In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from datetime import timedelta
plt.rcParams['figure.figsize'] = [11.69, 8.27]
plt.rcParams['font.size'] = 17

In [ ]:
def price():
    
    start_time = time.monotonic()

    # Monte Carlo matrixes
    
    if antithetic_variates == "off":
        
        np.random.seed(23)
        mc_ir = np.random.standard_normal((M,N))
        np.random.seed(32)
        mc_stock = np.random.standard_normal((M,N))
    
    if antithetic_variates == "on":
        
        np.random.seed(23)
        mc_ir = np.zeros((M,int(N*2)))
        mc_ir[:,:N] = np.random.standard_normal((M,N))
        mc_ir[:,N:] = -mc_ir[:,:N]    
        np.random.seed(32)
        mc_stock = np.zeros((M,int(N*2)))
        mc_stock[:,:N] = np.random.standard_normal((M,N))
        mc_stock[:,N:] = -mc_stock[:,:N]
        
    # Interest Rates Simulations
    
    if antithetic_variates == "off":
        
        interest_rates = np.zeros((M,N))
        interest_rates[0,:] = r0
        
        for i in range(0,N):
            for j in (range(1,M)):
                interest_rates[j,i] = interest_rates[j-1,i]\
                                     + kappa*(theta - interest_rates[j-1,i])* dt\
                                     + sigma_ir* np.sqrt(dt) * mc_ir[j,i]  
        
    if antithetic_variates == "on":
        
        interest_rates = np.zeros((M,N*2))
        interest_rates[0,:] = r0
               
        for i in range(0,N*2):
            for j in (range(1,M)):
                interest_rates[j,i] = interest_rates[j-1,i]\
                                     + kappa*(theta - interest_rates[j-1,i])* dt\
                                     + sigma_ir* np.sqrt(dt) * mc_ir[j,i]

    # Interest Rate Curves
    
    if antithetic_variates == "off":
        
        interest_rate_curve = np.zeros((M,N))
        interest_rate_curve[0,:] = r0 * dt

        for i in range(0,N):
            for j in (range(1,M)):
                interest_rate_curve[j,i] = interest_rate_curve[j-1,i]\
                                            + interest_rates[j,i] * dt
        
    if antithetic_variates == "on":
        
        interest_rate_curve = np.zeros((M,N*2))
        interest_rate_curve[0,:] = r0 * dt

        for i in range(0,N*2):
            for j in (range(1,M)):
                interest_rate_curve[j,i] = interest_rate_curve[j-1,i]\
                                            + interest_rates[j,i] * dt
    
    # Stock Price Simulations
    
    if antithetic_variates == "off":
        
        stock_prices = np.zeros((M,N))
        stock_prices[0,:] = S0
        
        for i in range(0,N):
            for j in (range(1,M)):
                stock_prices[j,i] = stock_prices[j-1,i] * np.exp( (interest_rates[j-1,i]\
                                    - sigma_stock**2 / 2) * dt\
                                    + sigma_stock * np.sqrt(dt) * (ir_correlation * mc_ir[j,i]\
                                    + np.sqrt(1 - ir_correlation**2) * mc_stock[j,i]))
        
    if antithetic_variates == "on":
        
        stock_prices = np.zeros((M,N*2))
        stock_prices[0,:] = S0
        
        for i in range(0,N*2):
            for j in (range(1,M)):
                stock_prices[j,i] = stock_prices[j-1,i] * np.exp( (interest_rates[j-1,i]\
                                    - sigma_stock**2 / 2) * dt\
                                    + sigma_stock * np.sqrt(dt) * (ir_correlation * mc_ir[j,i]\
                                    + np.sqrt(1 - ir_correlation**2) * mc_stock[j,i]))
    
    # Payoff (discounted)

    if payoff == "Asian":
        
        if antithetic_variates == "off":

            disc_payoff = np.zeros(N)
            if option == "Call":
                for i in range(0,N):
                    if stock_prices[:,i].mean() - K > 0.:
                        disc_payoff[i] =  (stock_prices[:,i].mean() - K) * np.exp((-interest_rate_curve[-1,i]))
            if option == "Put":
                for i in range(0,N):
                    if K - stock_prices[:,i].mean() > 0.:
                        disc_payoff[i] =  (K - stock_prices[:,i].mean()) * np.exp((-interest_rate_curve[-1,i]))

        if antithetic_variates == "on":

            disc_payoff = np.zeros(N*2)
            if option == "Call":
                for i in range(0,N*2):
                    if stock_prices[:,i].mean() - K > 0.:
                        disc_payoff[i] =  (stock_prices[:,i].mean() - K) * np.exp((-interest_rate_curve[-1,i]))
            if option == "Put":
                for i in range(0,N*2):
                    if K - stock_prices[:,i].mean() > 0.:
                        disc_payoff[i] =  (K - stock_prices[:,i].mean()) * np.exp((-interest_rate_curve[-1,i]))
            new_disc_payoff = np.zeros(N)
            for i in range(0,N):
                new_disc_payoff[i] = (disc_payoff[i] + disc_payoff[i+N])/2

            disc_payoff = new_disc_payoff
            
    if payoff == "Vanilla":
        
        if antithetic_variates == "off":
            disc_payoff = np.zeros(N)

            if option == "Call":
                for i in range(0,N):
                    if stock_prices[-1,i] - K > 0.:
                        disc_payoff[i] =  (stock_prices[-1,i] - K) * np.exp((-interest_rate_curve[-1,i]))

            if option == "Put":
                for i in range(0,N):
                    if K - stock_prices[-1,i] > 0.:
                        disc_payoff[i] =  (K - stock_prices[-1,i]) * np.exp((-interest_rate_curve[-1,i]))


        if antithetic_variates == "on":
            disc_payoff = np.zeros(N*2)

            if option == "Call":
                for i in range(0,N*2):
                    if stock_prices[-1,i] - K > 0.:
                        disc_payoff[i] =  (stock_prices[-1,i] - K) * np.exp((-interest_rate_curve[-1,i]))

            if option == "Put":
                for i in range(0,N*2):
                    if K - stock_prices[-1,i] > 0.:
                        disc_payoff[i] =  (K - stock_prices[-1,i]) * np.exp((-interest_rate_curve[-1,i]))

            new_disc_payoff = np.zeros(N)
            for i in range(0,N):
                new_disc_payoff[i] = (disc_payoff[i] + disc_payoff[i+N])/2

            disc_payoff = new_disc_payoff  
    
    # Results
    
    price = disc_payoff.sum() / N
    
    if print_option == "on":
    
        if antithetic_variates == "off":
            print("Using brute Monte Carlo")

        if antithetic_variates == "on":
            print("Using antithetic variates,")

        print("the price estimate of the "+str(payoff)+" "+str(option)+" Option is: "+str(price)+" ,"\
              +"\nits confidence interval is: ["+str(price - 1.96*disc_payoff.std()/N**0.5)+\
              "  ; "+str(price+1.96*disc_payoff.std()/N**0.5)+"]")

    #plots
    
    if plots == "on":
        
        plt.title("Interest Rate Simulations")
        plt.ylabel("Interest Rate")
        plt.xlabel("Maturity (months)")
        for i in range(interest_rates.shape[1]):
            plt.plot(interest_rates[:,i])
        plt.show()
        
        plt.title("Interest Rate Simulations")
        plt.ylabel("Interest Rate")
        plt.xlabel("Maturity (months)")
        for i in range(0,10):
            plt.plot(interest_rates[:,i])
        plt.plot(interest_rates.mean(axis=1), color = "b", linewidth = 3, linestyle = "-.", label = "average level")
        plt.legend()
        plt.show()
        
        plt.title("Interest Rate Curve Integral Construction")
        plt.ylabel("Interest Rate")
        plt.xlabel("Maturity (months)")
        plt.plot(interest_rates[:,0], label = "Interest Rate")
        plt.plot(interest_rate_curve[:,0], label = "IRC")
        plt.legend()
        plt.show()
        
        plt.title("Interest Rate Curve Simulations")
        plt.ylabel("Interest Rate Curve")
        plt.xlabel("Maturity (months)")
        for i in range(interest_rate_curve.shape[1]):
            plt.plot(interest_rate_curve[:,i])
        plt.show()
        
        plt.title("Value of the Money Market Account")
        plt.ylabel("B(t)")
        plt.xlabel("Maturity (months)")
        for i in range(interest_rate_curve.shape[1]):
            plt.plot(np.exp(interest_rate_curve[:,i]))
        plt.show()
        
        plt.title("Stock Price Simulations")
        plt.ylabel("Stock Price")
        plt.xlabel("Maturity (months)")
        for i in range(stock_prices.shape[1]):
            plt.plot(stock_prices[:,i])
        plt.show()
        
        plt.title("Stock Price Simulations")
        plt.ylabel("Stock Price")
        plt.xlabel("Maturity (months)")
        for i in range(0,10):
            plt.plot(stock_prices[:,i])
        plt.plot(stock_prices.mean(axis=1), color = "b", linewidth = 3, linestyle = "-.", label = "average price")
        plt.legend()
        plt.show()
        
    end_time = time.monotonic()
    
    if print_option == "on":
        print("the time taken is: "+str(timedelta(seconds=end_time - start_time)))
    
    return price

Inputs

In [ ]:
# maturity of the option (in years)
T = 1/2

# m number of monitoring dates
M = int(360/2)

# discretization time
dt = T/M

# number of simulation
N = 300000

# list of discretization dates, tao
tao = np.linspace(0,T,M) 

# antithetic variates variance reduction ("on" or "off")
antithetic_variates = "on"

# ir vriables
r0 = -0.00732515
kappa = 0.0271933
theta = 0.06032162
sigma_ir = 0.0118128

# stock variables
S0 = 100
sigma_stock = 0.1
ir_correlation = 0.05

# strike price K
K = 100

# select option type, "Call" or "Put"
option = "Call"

# payoff, "Vanilla" or "Asian"
payoff = "Asian"

# plots, "on" or "off"
plots = "off"

# print results, "on" or "off"
print_option = "on"

Run the function!

In [ ]:
price()